# Seq2Seq

*Sequence to Sequence*.  
*Encoder-Decoder Model*とも。

2つのモデルを用いて、入力されたシーケンスに基づいた別のシーケンスを出力するモデル。

これまで、RNN（やLSTM）を用いて、入力した単語列に続く単語を予測するモデルを作成し、単語の予測を繰り返すことで文章を生成した。

RNNは時系列の情報を保持するために隠れ状態$h_t$を用いる。隠れ状態の初期値$h_0$は0ベクトルとしているが、ここで、何らかの入力データから生成したベクトルを$h_0$として用いることを考える。このとき、上手く学習させれば、その入力に基づいた文章を生成できるのではないか。

例えば、入力を画像をとし、CNNを用いて抽出した特徴量を$h_0$として用いるようにすれば、入力画像に基づいた文章が生成できる。画像のキャプション生成などに応用できそう。隠れ状態を通じてRNNからCNNまで逆伝播が繋がるので、画像と文章のペアさえ用意すれば学習できそう。というかできる[1]。

[1] [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/abs/1411.4555)

では、入力に文章を用いることはできないだろうか。RNNに文章を入力し、最後に出力された隠れ状態を文章ベクトルとする。これを別のRNNへの入力$h_0$とすれば、入力文に基づいた文章生成が可能になる。

この発想は翻訳タスクに大きく役立つ。入力と出力に同じ意味を持った異なる言語の文章を設定すれば、入力文と同じ意味を持った文章生成が可能になる。

本章ではこの翻訳モデルを作成する。入力に日本語文、出力に入力と同じ意味を持つ英語文を設定し、日本語→英語の翻訳を行うモデルを作成する。こういった、シーケンスをシーケンスに変換するモデルを *Seq2Seq (Sequence to Sequence)* と呼ぶ。

Seq2Seqは以下の二つのRNNから構成される。

- ***Encoder*** : 文章ベクトルを生成するRNN
- ***Decoder*** : 文章ベクトルを受け取って出力文を生成するRNN

このことから***Encoder-Decoderモデル***とも呼ばれる。

In [1]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings; warnings.filterwarnings("ignore")
import math
import random

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(
    width=20,
    with_test=True,
    label="ppl train",
    round=2,
    agg_fn=lambda s, w: math.exp(s / w)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## 対訳コーパス

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットから日本語と英語の対訳コーパスを使用する。

- [iwslt2017  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/community_catalog/huggingface/iwslt2017?hl=ja#iwslt2017-en-ja)

In [3]:
ds = tfds.load(
    "huggingface:iwslt2017/iwslt2017-en-ja",
    data_dir="data",
    split="train"
)
ds = list(ds.as_numpy_iterator())

In [4]:
data_ja = []
data_en = []
for sample in ds:
    ja = sample["translation"]["ja"].decode()
    en = sample["translation"]["en"].decode()
    data_ja.append(ja)
    data_en.append(en)

print("num of data:", len(data_ja), "\n")
for _ in range(5):
    i = random.randint(0, len(data_ja))
    print(data_ja[i])
    print(data_en[i])
    print()

num of data: 223108 

ここから導き出される重要な結論は 私たちは齧歯類ではなく
So this brings us to a very important conclusion already, which is that we are not rodents.

そこにいた14匹のオカビの1匹は妊娠していました
We have a zoo of 14 Okapis, and one of them was pregnant.

若者が好むようなものは 何もないのです 車も 女性も テレビも 何もありません あるのは戦闘のみです
that young men typically like: no cars, no girls, no television, nothing except combat.

あるいは 大きな規模で 重力や一般相対性理論の修正を考えたり 私たちの宇宙が数ある宇宙の一つ つまり この神秘的な多元宇宙の一部だ と言う人たちもいます これらの考え方や理論のすべてが 素晴らしく ―幾分クレージーなものもありますが― いずれも42個の点と 整合しています
Or, they look at large scales and change how gravity and general relativity work, or they say our universe is just one of many, part of this mysterious multiverse, but all of these ideas, all of these theories, amazing and admittedly some of them a little crazy, but all of them consistent with our 42 points.

ゆっくり後ろに下がると ゴッホが描いた「煙草を吸う骸骨」が見えます
And, as you slowly step back, you see that it's a painting by Van Gogh, called "Skull with Cigarette."



書き出し

In [5]:
textfile_ja = "data/iwslt2017_ja.txt"
with open(textfile_ja, "w") as f:
    f.write("\n".join(data_ja))

textfile_en = "data/iwslt2017_en.txt"
with open(textfile_en, "w") as f:
    f.write("\n".join(data_en))

読み込み

In [6]:
textfile_ja = f"data/iwslt2017_ja.txt"
textfile_en = f"data/iwslt2017_en.txt"

with open(textfile_ja) as f:
    data_ja = f.read().splitlines()
with open(textfile_en) as f:
    data_en = f.read().splitlines()

### 前処理

日本語、英語別々にトークナイザを作成する。

In [7]:
tokenizer_prefix_ja = f"models/tokenizer_iwslt2017_ja"
tokenizer_prefix_en = f"models/tokenizer_iwslt2017_en"
pad_id = 3
vocab_size = 8000

In [ ]:
spm.SentencePieceTrainer.Train(
    input=textfile_ja,
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size,
    pad_id=pad_id
)

spm.SentencePieceTrainer.Train(
    input=textfile_en,
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size,
    pad_id=pad_id
)

In [10]:
sp_ja = spm.SentencePieceProcessor(f"{tokenizer_prefix_ja}.model")
sp_en = spm.SentencePieceProcessor(f"{tokenizer_prefix_en}.model")

unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
pad_id = sp_ja.pad_id()

n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print("num of vocabrary (ja):", n_vocab_ja)
print("num of vocabrary (en):", n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [11]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [12]:
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

### 学習データの作成

入力文と正解のペアを作成する。

Encoderへの入力（入力文）とDecoderの出力（正解）のペアを作成する。また、Decoderへの入力を考える必要がある。今回は教師強制を採用し、出力文の頭に\<BOS>を付与したものをDecoderへの入力とする。

例）
Encoderへの入力（入力文） | Decoderへの入力 | Decoderの出力（出力文）
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

`DataLoader`の作成。

In [13]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # Encoderへの入力
        x_dec = en[:-1] # Decoderへの入力
        y_dec = en[1:] # Decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
print("num of train data:", len(train_dataset))
print("num of test data:", len(test_dataset))

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

# example
x_enc, x_dec, y_dec = next(iter(train_loader))
x_enc.shape, x_dec.shape, y_dec.shape

num of train data: 178487
num of test data: 44621


(torch.Size([32, 75]), torch.Size([32, 90]), torch.Size([32, 90]))


---

## Seq2Seqを用いた翻訳モデル

Encoder、Decoderを作成し、Seq2Seqモデルを作成する。

### Encoder

入力文を入れて隠れ状態を出力するだけのRNN。LSTMと線形層で作る。

ここで、何を隠れ状態とするかを考える。Decoderに渡したい隠れ状態として満たしてほしい条件は以下である。

- （padを除いた）全ての入力を参照して出力されている
- 固定長

これらを全て満たしていれば何でもよい。

よくあるのは最後の隠れ状態。これが一番シンプルで実装も簡単。

ただこれだと学習時にpadトークンが隠れ状態に関与してしまう。多くの場合、推論時にpadトークンは存在しないため、学習時もpadトークンを考慮しない出力をさせた方が良さそう。また、padの数が多くなるにつれて隠れ状態がある一定の値に収束してしまう（経験談）。RNNに同じトークンを何度も入力することで隠れ状態が収束してしまうみたい。そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、入力文を考慮した出力が行えない。

ではどうするかというと、「padを除いた最後の隠れ状態」とする。eosトークンが入力された時点の隠れ状態とも言える。これで上記の問題を解決できる。

他にもいくつかの案が考えられる。例えば、「padを除いた全ての隠れ状態の平均」とか。これも条件を満たす。

また、RNNを双方向にするという手法もある。Encoderは入力文の特徴を抽出できれば良いので、文の初めから順に処理しなければならない訳ではない。例えば逆から処理してもよい。

双方向RNNでは、順方向と逆方向の双方向で演算を行い、各時刻で二つの隠れ状態を出力する。両端の時刻の隠れ状態を取得することで二種類の隠れ状態が得られ、それらを足すないしは結合することで最終的な一つの隠れ状態が得られる。

双方向RNNは`torch.nn.RNN`の引数`bidirectional=True`で実装できる。ただ、各隠れ状態の出力がpadトークンを除いて行われて欲しいという望みがあり、それを実現するための実装が難しそうなので、今回は不採用。

結局どうしようかという感じだが、無難に「padを除いた最後の隠れ状態」を採用する。Encoderを以下のように実装する。

In [14]:
class Encoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_pos = x == eos_id
            # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, _ = self.lstm(x) # (batch_size, seq_len, hidden_size)
        h = hs[eos_pos] # (batch_size, hidden_size)
        h = self.dropout(h)
        h = self.fc(h) # (batch_size, hidden_size)
        return h

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、LSTMと線形層で作る。

In [15]:
class Decoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, hc):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, hc = self.lstm(x, hc) # (batch_size, seq_len, hidden_size)
        hs = self.dropout(hs)
        y = self.fc(hs) # (batch_size, seq_len, n_vocab)
        return y, hc

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        hc = self.get_hc(h)
        y, _ = self.decoder(x_dec, hc)
        return y

    def get_hc(self, h):
        h = h.unsqueeze(0)
        c = torch.zeros_like(h)
        hc = (h, c)
        return hc

In [17]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 16,761,152



---

## 実践

### 学習

In [18]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model):
    model.eval()
    losses = []
    for x_enc, x_dec, y_dec in test_loader:
        x_enc = x_enc.to(device)
        x_dec = x_dec.to(device)
        y_dec = y_dec.to(device)

        y = model(x_enc, x_dec)
        loss = loss_fn(y, y_dec)
        losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = math.exp(loss)
    return ppl

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [19]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train(model, optimizer, n_epochs=20, prog_unit=1)

In [22]:
model_path = "models/lm_seq2seq.pth"
torch.save(model.state_dict(), model_path)

### 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。

In [24]:
def token_sampling(y, decisive=True):
    y.squeeze_(0)
    if decisive:
        token = y.argmax().item()
    else:
        y[unk_id] = -torch.inf
        probs = F.softmax(y, dim=-1)
        token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device).unsqueeze(0)

    h_enc = model.encoder(in_ids)
    hc = model.get_hc(h_enc)
    next_token = bos_id

    token_ids = []
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device).reshape(1, 1)
        y, hc = model.decoder(x, hc)
        next_token = token_sampling(y, decisive)
        token_ids.append(next_token)

    sentence = sp_en.decode(token_ids)
    return sentence

まずは訓練データから。

In [25]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: かいつまんで話すと そのお方が事務所に来ました 良い身なりでしたよ


output: I was a little girl who was a very, very small man who was a very good friend.
answer: To make a long story short, the gentleman comes into the office, great suit and tie.

input: 企業の行動は 単純に社会貢献活動や 寄付金を増やす事で変わりません
output: The other thing that we're doing is to make a difference, and that's the problem of the human being, and that's what we need to do.
answer: Now, we're not going to change corporate behavior by simply increasing corporate philanthropy or charitable contributions.

input: もし4つ全てが 国家の問題だと意識する方は
output: If you're a leader, you're not going to be a global citizen.
answer: Please raise your hand right now if you're willing to admit that all four of these are national problems.

input: 勿論 法律や都市も 人にとって便利です
output: But, of course, is the most important, the most important, the most important.
answer: But it's also law. And of course cities are ways to make things more useful to us.

input: イギリスでは 63%の 男性の短期受刑者が 出所後 1年以内に 再犯をしています
output: In the United States, the avera

訓練データに含まれていないものも試してみる。

In [26]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: このプロセスで研究者は 意図的にミツバチを選別し交配できますが これには代償が伴います


output: And this is the same thing, and the same thing is going to be the same thing, and it's a very small group of people, and they're all going to do this.
answer: Now, this procedure allows the researchers to control exactly which bees are being crossed, but there's a tradeoff in having this much control.

input: 私の研究室では1年以上も ワクチンを23°Cで保管して 活性が失われないことを示しました
output: In the last few years, I was a very small group of people who were able to get a lot of money, and they were able to get a lot of money.
answer: Within my lab we've shown that we can keep the vaccine stored at 23 degrees Celsius for more than a year without any loss in activity at all.

input: 雷に打たれたかのようでした
output: It was a little bit like a sympathy.
answer: And it was like a lightning bolt.

input: 人工的に作られたワクチンを 人々に投与するのは その利点の方が危険より 勝ると思われるからです
output: Because we're not able to do that, but they're not going to be able to do it, and they're going to have a lot of money.
answer: When we put vaccines into people, we are

In [27]:
# original
sentences = [
    "ありがとう。",
    "猫はかわいいね。",
    "上手く文章が書けるようになりました。"
]

for sentence in sentences:
    print("input:", sentence)
    print("output:", translate(model, sentence))
    print()

input: ありがとう。
output: Thank you.

input: 猫はかわいいね。
output: The water is not the same.

input: 上手く文章が書けるようになりました。
output: I was a little bit of a woman.

